In [30]:
import pandas as pd
import numpy as np
import csv as csv
import glob
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [31]:
#find all csv files in a directory
csv_df =[]
for file in glob.glob("resources/*.csv"):
    csv = pd.read_csv(file, header=0)
    csv_df.append(csv)
    #print (files) 
    
train_df = pd.concat(csv_df)
print(train_df.shape)
train_df.head()

(166927, 7)


,frame.number,frame.time,ip.src,ip.dst,ip.proto,frame.len,_ws.col.Info
0,1,"Apr 22, 2011 13:23:49.238845000 Central Daylig...",1.1.23.3,1.1.12.1,6,60,"46557→80 [SYN, ECN, CWR] Seq=0 Win=4128 Len=0 ..."
1,2,"Apr 22, 2011 13:23:49.609845000 Central Daylig...",1.1.12.1,1.1.23.3,6,58,"80→46557 [SYN, ACK, ECN] Seq=0 Ack=1 Win=4128 ..."
2,3,"Apr 22, 2011 13:23:49.690845000 Central Daylig...",1.1.23.3,1.1.12.1,6,60,46557→80 [ACK] Seq=1 Ack=1 Win=4128 Len=0
3,4,"Apr 22, 2011 13:23:49.700845000 Central Daylig...",1.1.23.3,1.1.12.1,6,215,GET /show-tech HTTP/1.1
4,5,"Apr 22, 2011 13:23:50.151845000 Central Daylig...",1.1.12.1,1.1.23.3,6,310,[TCP segment of a reassembled PDU]


In [32]:
#extracting additional features
flags = ["syn", "ack", "fin", "push", "urgent", "unreachable"]

def extract_feature(dt, feature):
    if feature in dt.lower():
        return 1
    else:
        return 0
    
for flag in flags:
    train_df[flag] = np.vectorize(extract_feature)(train_df['_ws.col.Info'], flag)
       
train_df.head()

,frame.number,frame.time,ip.src,ip.dst,ip.proto,frame.len,_ws.col.Info,syn,ack,fin,push,urgent,unreachable
0,1,"Apr 22, 2011 13:23:49.238845000 Central Daylig...",1.1.23.3,1.1.12.1,6,60,"46557→80 [SYN, ECN, CWR] Seq=0 Win=4128 Len=0 ...",1,0,0,0,0,0
1,2,"Apr 22, 2011 13:23:49.609845000 Central Daylig...",1.1.12.1,1.1.23.3,6,58,"80→46557 [SYN, ACK, ECN] Seq=0 Ack=1 Win=4128 ...",1,1,0,0,0,0
2,3,"Apr 22, 2011 13:23:49.690845000 Central Daylig...",1.1.23.3,1.1.12.1,6,60,46557→80 [ACK] Seq=1 Ack=1 Win=4128 Len=0,0,1,0,0,0,0
3,4,"Apr 22, 2011 13:23:49.700845000 Central Daylig...",1.1.23.3,1.1.12.1,6,215,GET /show-tech HTTP/1.1,0,0,0,0,0,0
4,5,"Apr 22, 2011 13:23:50.151845000 Central Daylig...",1.1.12.1,1.1.23.3,6,310,[TCP segment of a reassembled PDU],0,0,0,0,0,0


In [33]:
#specifying protocol for later calculation
def protocol(dt, index):
    if dt == index:
        return 1
    else:
        return 0
    
train_df.insert(5, 'tcp', np.vectorize(protocol)(train_df['ip.proto'], 6))
train_df.insert(6, 'icmp', np.vectorize(protocol)(train_df['ip.proto'], 1))

train_df.head()

,frame.number,frame.time,ip.src,ip.dst,ip.proto,tcp,icmp,frame.len,_ws.col.Info,syn,ack,fin,push,urgent,unreachable
0,1,"Apr 22, 2011 13:23:49.238845000 Central Daylig...",1.1.23.3,1.1.12.1,6,1,0,60,"46557→80 [SYN, ECN, CWR] Seq=0 Win=4128 Len=0 ...",1,0,0,0,0,0
1,2,"Apr 22, 2011 13:23:49.609845000 Central Daylig...",1.1.12.1,1.1.23.3,6,1,0,58,"80→46557 [SYN, ACK, ECN] Seq=0 Ack=1 Win=4128 ...",1,1,0,0,0,0
2,3,"Apr 22, 2011 13:23:49.690845000 Central Daylig...",1.1.23.3,1.1.12.1,6,1,0,60,46557→80 [ACK] Seq=1 Ack=1 Win=4128 Len=0,0,1,0,0,0,0
3,4,"Apr 22, 2011 13:23:49.700845000 Central Daylig...",1.1.23.3,1.1.12.1,6,1,0,215,GET /show-tech HTTP/1.1,0,0,0,0,0,0
4,5,"Apr 22, 2011 13:23:50.151845000 Central Daylig...",1.1.12.1,1.1.23.3,6,1,0,310,[TCP segment of a reassembled PDU],0,0,0,0,0,0


In [34]:
def time(dt):
    date_and_time = dt.split('.')
    return datetime.strptime(date_and_time[0], "%b %d, %Y %H:%M:%S")
train_df.insert(2, 'time', train_df['frame.time'].map(time))
train_df.head()

,frame.number,frame.time,time,ip.src,ip.dst,ip.proto,tcp,icmp,frame.len,_ws.col.Info,syn,ack,fin,push,urgent,unreachable
0,1,"Apr 22, 2011 13:23:49.238845000 Central Daylig...",2011-04-22 13:23:49,1.1.23.3,1.1.12.1,6,1,0,60,"46557→80 [SYN, ECN, CWR] Seq=0 Win=4128 Len=0 ...",1,0,0,0,0,0
1,2,"Apr 22, 2011 13:23:49.609845000 Central Daylig...",2011-04-22 13:23:49,1.1.12.1,1.1.23.3,6,1,0,58,"80→46557 [SYN, ACK, ECN] Seq=0 Ack=1 Win=4128 ...",1,1,0,0,0,0
2,3,"Apr 22, 2011 13:23:49.690845000 Central Daylig...",2011-04-22 13:23:49,1.1.23.3,1.1.12.1,6,1,0,60,46557→80 [ACK] Seq=1 Ack=1 Win=4128 Len=0,0,1,0,0,0,0
3,4,"Apr 22, 2011 13:23:49.700845000 Central Daylig...",2011-04-22 13:23:49,1.1.23.3,1.1.12.1,6,1,0,215,GET /show-tech HTTP/1.1,0,0,0,0,0,0
4,5,"Apr 22, 2011 13:23:50.151845000 Central Daylig...",2011-04-22 13:23:50,1.1.12.1,1.1.23.3,6,1,0,310,[TCP segment of a reassembled PDU],0,0,0,0,0,0


In [35]:
def add_label(label):
    train_df['label'] = label

add_label("attack")
train_df.head()

,frame.number,frame.time,time,ip.src,ip.dst,ip.proto,tcp,icmp,frame.len,_ws.col.Info,syn,ack,fin,push,urgent,unreachable,label
0,1,"Apr 22, 2011 13:23:49.238845000 Central Daylig...",2011-04-22 13:23:49,1.1.23.3,1.1.12.1,6,1,0,60,"46557→80 [SYN, ECN, CWR] Seq=0 Win=4128 Len=0 ...",1,0,0,0,0,0,attack
1,2,"Apr 22, 2011 13:23:49.609845000 Central Daylig...",2011-04-22 13:23:49,1.1.12.1,1.1.23.3,6,1,0,58,"80→46557 [SYN, ACK, ECN] Seq=0 Ack=1 Win=4128 ...",1,1,0,0,0,0,attack
2,3,"Apr 22, 2011 13:23:49.690845000 Central Daylig...",2011-04-22 13:23:49,1.1.23.3,1.1.12.1,6,1,0,60,46557→80 [ACK] Seq=1 Ack=1 Win=4128 Len=0,0,1,0,0,0,0,attack
3,4,"Apr 22, 2011 13:23:49.700845000 Central Daylig...",2011-04-22 13:23:49,1.1.23.3,1.1.12.1,6,1,0,215,GET /show-tech HTTP/1.1,0,0,0,0,0,0,attack
4,5,"Apr 22, 2011 13:23:50.151845000 Central Daylig...",2011-04-22 13:23:50,1.1.12.1,1.1.23.3,6,1,0,310,[TCP segment of a reassembled PDU],0,0,0,0,0,0,attack


In [36]:
train_df.describe()

,frame.number,ip.proto,tcp,icmp,frame.len,syn,ack,fin,push,urgent,unreachable
count,166927.000000,166927.000000,166927.00000,166927.000000,166927.000000,166927.000000,166927.000000,166927.000000,166927,166927,166927.000000
mean,82986.374499,3.133136,0.42657,0.573412,222.757325,0.056108,0.439575,0.037993,0,0,0.016247
std,48185.451406,2.473583,0.49458,0.494583,420.435651,0.230132,0.496337,0.191179,0,0,0.126423
min,1.000000,1.000000,0.00000,0.000000,40.000000,0.000000,0.000000,0.000000,0,0,0.000000
25%,41253.500000,1.000000,0.00000,0.000000,60.000000,0.000000,0.000000,0.000000,0,0,0.000000
50%,82985.000000,1.000000,0.00000,1.000000,60.000000,0.000000,0.000000,0.000000,0,0,0.000000
75%,124716.500000,6.000000,1.00000,1.000000,60.000000,0.000000,1.000000,0.000000,0,0,0.000000
max,166448.000000,17.000000,1.00000,1.000000,1500.000000,1.000000,1.000000,1.000000,0,0,1.000000
